In [4]:
import pandas as pd
import healpy as hp
import numpy as np
import hipscat as hc
import tempfile



In [25]:
%%time

file_name = "/data3/epyc/data3/hipscat/catalogs/ztf_dr14_explode/Norder1/Npix33/source_hipscat_id.parquet"

keep_columns = ["_hipscat_id", "ps1_objid"]

data_frame = pd.read_parquet(file_name, engine="pyarrow", columns=keep_columns)

data_frame['_hipscat_id'] = data_frame.index
data_frame = data_frame.set_index("ps1_objid").sort_index()

data_frame
tmp = "/data3/epyc/data3/hipscat/tmp/tmp_ztf_index/"

data_frame.to_parquet((tmp+"source_index_part.parquet"))

CPU times: user 9.17 s, sys: 7.86 s, total: 17 s
Wall time: 16.1 s


In [31]:
primary_name = "/data3/epyc/data3/hipscat/catalogs/ztf_mar16/object_index/parts=0/part_000_of_001.parquet"
join_name = "/data3/epyc/data3/hipscat/catalogs/ztf_mar16/source_index/parts=0/part_000_of_001.parquet"


primary_frame = pd.read_parquet(primary_name, engine="pyarrow")
primary_frame['primary_id'] = primary_frame.index
primary_frame['primary_hipscat_id'] = primary_frame['_hipscat_id']

join_frame = pd.read_parquet(join_name, engine="pyarrow")
join_frame['join_hipscat_id'] = join_frame['_hipscat_id']
join_frame['join_id'] = join_frame.index



# full_frame = primary_frame.merge(join_frame, on="ps1_objid")


In [33]:
%%time

full_frame = primary_frame.merge(join_frame, left_on="primary_id", right_on="join_id")
full_frame = full_frame[['primary_id', 'primary_hipscat_id', 'join_id', 'join_hipscat_id']]
full_frame

tmp = "/data3/epyc/data3/hipscat/tmp/tmp_ztf_index/"

full_frame.to_parquet((tmp+"association.parquet"), index=False)

CPU times: user 10.7 s, sys: 17.6 s, total: 28.4 s
Wall time: 28.6 s


In [37]:
primary_name = "/data3/epyc/data3/hipscat/catalogs/ztf_mar16/object/partition_info.csv"
join_name = "/data3/epyc/data3/hipscat/catalogs/ztf_mar16/source/partition_info.csv"

primary_frame = pd.read_csv(primary_name).rename(columns={"order":"primary_Norder", "pixel":"primary_Npix"})

join_frame = pd.read_csv(join_name).rename(columns={"order":"join_Norder", "pixel":"join_Npix"})

full_frame = primary_frame.merge(join_frame, how="cross")
full_frame = full_frame[['primary_Norder', 'primary_Npix', 'join_Norder', 'join_Npix']]
full_frame.to_csv("/data3/epyc/data3/hipscat/catalogs/ztf_mar16/object_to_source/partition_join_info.csv", index=False)